In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
figure(num=None, figsize=(20, 10), dpi=80, facecolor='w', edgecolor='k')
import plotly.express as px
import plotly.graph_objs as go
import ppscore as pps
import seaborn as sns

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
label_file_path = "../input/us_census_full/labels.txt"
with open(label_file_path) as f:
    names = [name.strip().replace(' ', '_') for name in f.readlines()]

In [ ]:
train_path = "../input/us_census_full/census_income_learn.csv"
test_path = "../input/us_census_full/census_income_test.csv"
train = pd.read_csv(train_path, names=names, sep=", ")
test = pd.read_csv(test_path, names=names, sep=", ")

In [ ]:
print("Train data size : " + str(train.shape))
print("Test data size : " + str(test.shape))

In [ ]:
train.sample(5)

In [ ]:
train.sample(5)

In [ ]:
train["income"].loc[(train["income"] == "50000+.")] = 1
train["income"].loc[(train["income"] == "- 50000.")] = 0
test["income"].loc[(test["income"] == "50000+.")] = 1
test["income"].loc[(test["income"] == "- 50000.")] = 0

In [ ]:
col = "income"
ratio_count = pd.DataFrame(train[col].value_counts(dropna=False))
ratio_count["ratio"] = ratio_count/ratio_count.sum()
ratio_count["counts"] = ratio_count[col]
ratio_count[col] = ratio_count.index
ratio_count.sort_values('ratio',ascending=False,inplace=True)
fig = px.bar(ratio_count, x=col, y='counts')
fig.show()
display(ratio_count)

## Verifying objects type data

In [ ]:
def get_objects_ratios(df):
    for col in df.columns:
        dtype = str(df[col].dtype)
        if dtype == "object":
            print(col)
            ratio_count = pd.DataFrame(train[col].value_counts(dropna=False))
            ratio_count["ratio"] = ratio_count/ratio_count.sum()
            ratio_count["counts"] = ratio_count[col]
            ratio_count[col] = ratio_count.index
            ratio_count.sort_values('ratio',ascending=False,inplace=True)
            fig = px.bar(ratio_count, x=col, y='counts')
            fig.show()
            display(ratio_count)

class_of_worker -> categorical<br/>
education -> ordinal?<br/>
enroll_in_edu_inst_last_wk -> categorical<br/>
marital_stat -> categorical<br/>
major_industry_code -> categorical
major_occupation_code -> categorical<br/>
race -> categorical<br/>
hispanic_origin -> categorical<br/>
sex -> categorical<br/>
member_of_a_labor_union -> categorical<br/>
reason_for_unemployment -> categorical<br/>
full_or_part_time_employment_stat -> categorical<br/>
tax_filer_status -> categorical<br/>
region_of_previous_residence -> categorical<br/>
state_of_previous_residence -> categorical, **Not in universe & ?**<br/>
detailed_household_and_family_stat -> categorical<br/>
migration_code-change_in_msa -> categorical, **Not in universe & Not identifiable & ?**<br/>
migration_code-change_in_reg -> categorical, **Not in universe & ?**<br/>
migration_code-move_within_reg -> categorical, **Not in universe & ?**<br/>
live_in_this_house_1_year_ago -> categorical<br/>
migration_prev_res_in_sunbelt -> categorical, **Not in universe & ?**<br/>
family_members_under_18 -> categorical<br/>
country_of_birth_father -> categorical<br/>
country_of_birth_mother -> categorical<br/>
country_of_birth_self -> categorical<br/>
citizenship -> categorical<br/>
fill_inc_questionnaire_for_veteran's_admin -> categorical<br/>
income -> categorical<br/>

In [ ]:
get_objects_ratios(train)

In [ ]:
get_objects_ratios(test)

In [ ]:
# replace 'Not in universe', 'Not identifiable' and '?' by null
train.replace("Not in universe", np.nan, inplace=True)
train.replace("Not identifiable", np.nan, inplace=True)
train.replace("?", np.nan, inplace=True)

In [ ]:
def missing_data(df):
    total = df.isnull().sum()
    percent = (df.isnull().sum()/df.shape[0]*100)
    missing_values = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in df.columns:
        dtype = str(df[col].dtype)
        types.append(dtype)
    missing_values["Types"] = types
    missing_values.sort_values('Total',ascending=False,inplace=True)
    return(missing_values)
missing_datas = missing_data(train)

In [ ]:
missing_datas

Columns with too much missing data:<br/>

In [ ]:
empty_cols = missing_datas[missing_datas["Percent"] >= 51].index
print(empty_cols)

## Verifying continuous data

In [ ]:
train.select_dtypes(exclude=["object"]).hist(bins=50, figsize=(20,15))
plt.tight_layout(pad=0.4)
plt.show()

### Correlation matrices

In [ ]:
plt.figure(figsize=(16,12))
sns.heatmap(train.corr(), cmap="Blues", linewidths=0.75, annot=True)

## Feature comparisons

In [ ]:
train_sup = train[train["income"] == 1]
train_inf = train[train["income"] == 0]

In [ ]:
def show_feature_comparison(col):
    train_sup_col = train_sup[col].value_counts()
    train_sup_col = pd.DataFrame({col:train_sup_col.index, 'count':train_sup_col.values})

    train_inf_col = train_inf[col].value_counts()
    train_inf_col = pd.DataFrame({col:train_inf_col.index, 'count':train_inf_col.values})

    pie_sup = go.Pie(  
       labels = train_sup_col[col],
       values = train_sup_col["count"],
       domain=dict(x=[0, 0.5]),
       name="Above 50k",
       hole = 0.5,
       marker = dict(colors=['violet', 'cornflowerblue'], line=dict(color='#000000', width=2))
    )

    pie_inf = go.Pie(  
       labels = train_inf_col[col],
       values = train_inf_col["count"],
       domain=dict(x=[0.5, 1.0]), 
       name="Below 50k",
       hole = 0.5,
       marker = dict(colors=['cornflowerblue', 'violet'], line=dict(color='#000000', width=2))
    )

    data = [pie_sup, pie_inf]

    layout = go.Layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        title=col + ' percentage from +50k vs -50k',
        annotations=[dict(text="Above 50k", x=0.18, y=0.5, font_size=15, showarrow=False),
                     dict(text="Below 50k", x=0.85, y=0.5, font_size=15, showarrow=False)]
    )

    fig = go.Figure(data=data, layout=layout)

    fig.show()

In [ ]:
show_feature_comparison("sex")

In [ ]:
show_feature_comparison("race")

In [ ]:
def feature_transform(df, empty_cols):
    
    # Droping empty column
    df.drop(empty_cols, axis=1, inplace=True)

    # Creating a categorical variable for age
    df["ageCat"] = ""
    df["ageCat"].loc[(df["age"] < 18)] = 'young'
    df["ageCat"].loc[(df["age"] >= 18) & (df["age"] < 56)] = 'mature'
    df["ageCat"].loc[(df["age"] >= 56)] = 'senior'
    
    # Creating a categorical variable for hispanic origin
    df["hispanicCat"] = 1
    df["hispanicCat"].loc[(df["hispanic_origin"] == "All other")] = 0
    df["hispanicCat"].loc[(df["hispanic_origin"].isna())] = 0
    
    # Creating a categorical variable to tell if the passenger is a Young/Mature/Senior male or a Young/Mature/Senior female
    df["SexCat"] = ""
    df["SexCat"].loc[(df["sex"] == "Male") & (df["age"] <= 21)] = "youngmale"
    df["SexCat"].loc[(df["sex"] == "Male") & ((df["age"] > 21) & (df["age"]) < 50)] = "maturemale"
    df["SexCat"].loc[(df["sex"] == "Male") & (df["age"] > 50)] = "seniormale"
    df["SexCat"].loc[(df["sex"] == "Female") & (df["age"] <= 21)] = "youngfemale"
    df["SexCat"].loc[(df["sex"] == "Female") & ((df["age"] > 21) & (df["age"]) < 50)] = "maturefemale"
    df["SexCat"].loc[(df["sex"] == "Female") & (df["age"] > 50)] = "seniorfemale"
    
    # Dropping unused columns from the feature set
    df.drop(["hispanic_origin"], axis=1, inplace=True)
    
    target = df['income']
    
    # Splitting categorical and numerical column dataframes
    categorical_df = df.select_dtypes(include=['object'])
    numeric_df = df.select_dtypes(exclude=['object'])
    
    # And then, storing the names of categorical and numerical columns.
    categorical_columns = list(categorical_df.columns)
    numeric_columns = list(numeric_df.columns)
    
    print("Categorical columns:\n", categorical_columns)
    print("\nNumeric columns:\n", numeric_columns)

    return df, target, categorical_columns, numeric_columns

In [ ]:
df, target, categorical_columns, numeric_columns = feature_transform(train, empty_cols)

In [ ]:
# Function responsible for checking our model's performance on the test data
def testSetResultsClassifier(classifier, x_test, y_test):
    predictions = classifier.predict(x_test)
    
    results = []
    f1 = f1_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    roc_auc = roc_auc_score(y_test, predictions)
    accuracy = accuracy_score(y_test, predictions)
    
    results.append(f1)
    results.append(precision)
    results.append(recall)
    results.append(roc_auc)
    results.append(accuracy)
    
    print("\n\n#---------------- Test set results (Best Classifier) ----------------#\n")
    print("F1 score, Precision, Recall, ROC_AUC score, Accuracy:")
    print(results)
    
    return results